In [27]:
import sys
import uuid
import math
import time
import random
import numpy as np
import pandas as pd

from Graph_Creator import *

from typing import *


In [28]:
# debugging
DEBUG=False 
# DEBUG=True

In [29]:
BATCH_SIZE=10 if not DEBUG else 10
EDGES=[50, 100, 200, 300, 400, 500] if not DEBUG else []

In [30]:
# type definitions
Solution = List[int]
Population = List[Solution]
Graph = List[Tuple[int, int]]


def five_number_summary(data: List[int]):
    """calculate min, max, 25th, 50th, and 75th percentiles"""
    p = np.percentile(data, [0, 25, 50, 75, 100], method='midpoint')
    return {
        "min": p[0],
        "l_quartile": p[1],
        "median": p[2],
        "u_quartile": p[3],
        "max": p[4],
    }

class Logger:
    def __init__(self) -> None:
        self.logs = []

    def log(self, l):
        self.logs.append(l)
    
    def clear(self):
        self.logs = []

    def save(self, filename: str):
        with open(filename, "w") as f:
            f.write(pd.json_normalize(self.logs).reset_index(drop=True).to_csv())

In [31]:
class GeneticAlgorithm:
    """The genetic algorithm from the textbook"""
    def __init__(self, graph: Graph = []) -> None:
        self.logger = Logger()
        self.TOTAL_VERTICES = 50  # total number of vertices in the graph
        self.GRAPH_SIZE = 100  # the number of edges in the graph
        self.MAX_FITNESS = 50  # the maximum value of the fitness function
        self.MAX_RUNNING_TIME = 45  # 45 seconds, algorithm will be terminated if it runs lnoger than this time
        self.MAX_GENERATIONS = 50  # maximum number of generations the algorithm will run for
        self.POPULATION_SIZE = 100  # the total number of solutions in each generation
        self.COLORS = 3  # the number of colors available

        self.graph = graph


    def generate_solution(self) -> Solution:
        return np.random.choice(self.COLORS, self.TOTAL_VERTICES)

    def generate_population(self) -> Population:
        return [self.generate_solution() for x in range(0, self.POPULATION_SIZE)]

    def generate_graph(self) -> Graph:
        gc = Graph_Creator()
        return gc.CreateGraphWithRandomEdges(self.GRAPH_SIZE)

    def get_weights(self, fitnesses: List[float]) -> List[float]:
        """compute the weight from fitness values, used in choosing parents to make child"""
        fitnesses_nonzero = np.array(fitnesses)
        total = np.sum(fitnesses_nonzero)
        prob: List[float] = fitnesses_nonzero / total
        return list(prob)

    def choose_parents(self, population: Population, fitnesses: List[float]) -> Tuple[Solution, Solution]:
        """Choose two parents from the <populaation>. Choise is weighted by <fitnesses>"""
        weights = self.get_weights(fitnesses)
        indices = np.random.choice(len(population), 2, False, weights)
        return population[indices[0]], population[indices[1]]

    def reproduce(self, parent1: Solution, parent2: Solution) -> Solution:
        """generate a new solution from two solutions"""
        split = random.randint(0, len(parent1))
        left = parent1[:split]
        right = parent2[split:]
        child: Solution = []
        child.extend(left)
        child.extend(right)
        return child

    def mutate(self, solution: Solution, prob: float = 0.1, repeat: int = 1) -> Solution:
        """Try to mutate the <solution> <repeat> times, with each mutation having a success rate of <prob>"""
        solution = solution.copy()
        for i in range(0, repeat):
            pos = random.randrange(0, len(solution))
            if random.random() < prob:
                solution[pos] += random.randrange(1, self.COLORS)
                solution[pos] %= self.COLORS
        return solution

    def get_invalid_vertices(self, solution: Solution):
        """find list of vertices that are invalid ie they are connected and have the same color"""
        invalid_vertices = set()
        for index, edge in enumerate(self.graph):
            vert0, vert1 = edge
            if solution[vert0] == solution[vert1]:
                invalid_vertices.add(vert0)
                invalid_vertices.add(vert1)
        return list(invalid_vertices)

    def calculate_fitness(self, solution: Solution) -> int:
        """calculate the fitness value of the <solution>"""
        invalid_vertices = self.get_invalid_vertices(solution)
        return self.TOTAL_VERTICES - len(invalid_vertices) + sys.float_info.epsilon

    def calculate_fitnesses(self, population: Population) -> List[int]:
        """calculate the fitness values of individuals in the <population>"""
        return [self.calculate_fitness(p) for p in population]

    def sort_population(self, population: Population, fitnesses: List[int]) -> Tuple[Population, List[int]]:
        """sort the <population> based on the individual's <fitness>"""
        zipped_pop = zip(fitnesses, population)
        sorted_pop = sorted(zipped_pop, key=lambda t: t[0])
        unzipped = list(zip(*sorted_pop))
        return list(unzipped[1]), list(unzipped[0])

    def get_top(self, population: Population, top: float | int = 1):
        """return the top <top> fraction individuals"""
        fitnesses = self.calculate_fitnesses(population)
        population, fitnesses = self.sort_population(population, fitnesses)
        split = top
        if split <= 1:
            split = math.floor(len(population) * top)
        return population[-split:]

    def run(self, population: Population, id: str = uuid.uuid1()):
        start_time = time.perf_counter()
        elapsed_time = 0
        generation = 0
        max_fitness = -1
        sorted_population = []
        
        while (
            max_fitness < self.MAX_FITNESS
            and elapsed_time < self.MAX_RUNNING_TIME
            and generation < self.MAX_GENERATIONS
        ):
            fitnesses = self.calculate_fitnesses(population)
            sorted_population, sorted_fitnesses = self.sort_population(population, fitnesses)
            max_fitness = sorted_fitnesses[-1]
            new_population = []
            
            self.logger.log({
                "id": id,
                "size": self.GRAPH_SIZE,
                "generation": generation,
                "fitness": five_number_summary(sorted_fitnesses), 
            })

            for i in range(0, self.POPULATION_SIZE):
                parent1, parent2 = self.choose_parents(sorted_population, sorted_fitnesses)
                child = self.reproduce(parent1, parent2)
                child = self.mutate(child, prob=0.1, repeat=1)
                new_population.append(child)
            
            population = new_population
            generation += 1
            elapsed_time = time.perf_counter() - start_time

        return {
            "max_fitness": max_fitness,
            "solution": sorted_population[-1],
            "graph": self.graph,
            "generations": generation,
            "size": self.GRAPH_SIZE,
            "id": id,
            "time": elapsed_time,
        }


In [32]:
logger = Logger()
for s in EDGES:
    print(f"num egdes = {s}")
    for i in range(0, BATCH_SIZE):
        ga = GeneticAlgorithm()
        ga.logger = logger
        ga.GRAPH_SIZE = s
        ga.graph = ga.generate_graph()
        res = ga.run(ga.generate_population(), id=f"{s:03d}:{i:03d}")
        print(res["max_fitness"])
logger.save("./output/results.csv")


num egdes = 50
41.0
42.0
41.0
41.0
43.0
41.0
45.0
44.0
43.0
39.0
num egdes = 100
31.0
30.0
28.0
37.0
28.0
33.0
36.0
30.0
30.0
35.0
num egdes = 200
17.0
19.0
19.0
19.0
21.0
16.0
19.0
22.0
17.0
19.0
num egdes = 300
13.0
10.0
13.0
13.0
13.0
12.0
10.0
13.0
11.0
12.0
num egdes = 400
10.0
9.0
8.0
8.0
9.0
7.0
10.0
7.0
8.0
10.0
num egdes = 500
7.0
2.0
5.0
3.0
3.0
3.0
5.0
7.0
6.0
5.0


In [33]:
class ThreeColor(GeneticAlgorithm):
    """modified genetic algorithm"""
    def __init__(self) -> None:
        super().__init__()

    def get_weights(self, fitnesses: List[float]) -> List[float]:
        fitnesses_scaled = np.array(fitnesses) / self.MAX_FITNESS
        fitnesses_square = fitnesses_scaled ** 2
        fitnesses_nonzero = fitnesses_square + sys.float_info.epsilon
        total = np.sum(fitnesses_nonzero)
        prob: List[float] = fitnesses_nonzero / total
        return list(prob)
    
    def mutate_invalid(self, solution: Solution, prob: float = 0.1) -> Solution:
        """Try to mutate the <solution> <repeat> times, with each mutation having a success rate of <prob>"""
        solution = solution.copy()
        invalid_vertices = self.get_invalid_vertices(solution)
        for i in invalid_vertices:
            if random.random() < prob:
                solution[i] = random.randrange(1, self.COLORS)
        return solution

    def run(self, population: Population, id: str = uuid.uuid1()):
        start_time = time.perf_counter()
        elapsed_time = 0
        generation = 0
        max_fitness = -1
        sorted_population = []

        while (
            max_fitness < self.MAX_FITNESS
            and elapsed_time < self.MAX_RUNNING_TIME
            and generation < self.MAX_GENERATIONS
        ):
            fitnesses = self.calculate_fitnesses(population)
            sorted_population, sorted_fitnesses = self.sort_population(population, fitnesses)
            max_fitness = sorted_fitnesses[-1]
            new_population: Solution = []
            
            self.logger.log({
                "id": id,
                "size": self.GRAPH_SIZE,
                "generation": generation,
                "fitness": five_number_summary(sorted_fitnesses), 
            })

            for i in range(0, self.POPULATION_SIZE//2):
                parent1, parent2 = self.choose_parents(sorted_population, sorted_fitnesses)
                child0 = self.reproduce(parent1, parent2)
                child1 = self.mutate_invalid(child0, prob=100/max_fitness**2)
                child2 = self.mutate(child0, prob=100/max_fitness**2, repeat=5)
                new_population.append(child1)
                new_population.append(child2)
            
            old_population = [self.mutate(parent, 0.5, 5) for parent in sorted_population[-self.POPULATION_SIZE//2:]]
            new_population.extend(sorted_population[-self.POPULATION_SIZE//2:])
            new_population.extend(old_population)
            population = self.get_top(new_population)
            generation += 1
            elapsed_time = time.perf_counter() - start_time

        return {
            "max_fitness": max_fitness,
            "solution": sorted_population[-1],
            "graph": self.graph,
            "generations": generation,
            "size": self.GRAPH_SIZE,
            "id": id,
            "time": elapsed_time,
        }

In [34]:
logger = Logger()
for s in EDGES:
    print(f"num egdes = {s}")
    for i in range(0, BATCH_SIZE):
        ga = ThreeColor()
        ga.logger = logger
        ga.GRAPH_SIZE = s
        ga.MAX_GENERATIONS = s
        ga.graph = ga.generate_graph()
        res = ga.run(ga.generate_population(), id=f"{s:03d}:{i:03d}")
        print(res["max_fitness"])
logger.save("./output/results_new.csv")


num egdes = 50
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
num egdes = 100
50.0
48.0
46.0
45.0
46.0
48.0
46.0
48.0
48.0
46.0
num egdes = 200
30.0
29.0
27.0
30.0
27.0
29.0
32.0
30.0
29.0
31.0
num egdes = 300
23.0
22.0
22.0
23.0
23.0
22.0
22.0
25.0
25.0
21.0
num egdes = 400
19.0
17.0
20.0
18.0
20.0
18.0
18.0
18.0
19.0
16.0
num egdes = 500
15.0
16.0
15.0
13.0
12.0
18.0
17.0
14.0
16.0
10.0


In [35]:
Testcases = Graph_Creator().ReadGraphfromCSVfile('./Testcases/50'), Graph_Creator().ReadGraphfromCSVfile('./Testcases/100'), Graph_Creator().ReadGraphfromCSVfile('./Testcases/200')

logger = Logger()
for i, t in enumerate(Testcases):
    print(f"Test Case {i}")
    for j in range(0, BATCH_SIZE):
        ga = GeneticAlgorithm()
        ga.logger = logger
        ga.GRAPH_SIZE = len(t)
        ga.graph = t
        res = ga.run(ga.generate_population(), id=f"{len(t):03d}:{j:03d}")
        print(res["max_fitness"])
logger.save("./output/test.csv")

logger = Logger()
for i, t in enumerate(Testcases):
    print(f"Test Case {i}")
    for j in range(0, BATCH_SIZE):
        ga = ThreeColor()
        ga.logger = logger
        ga.GRAPH_SIZE = len(t)
        ga.MAX_GENERATIONS = len(t)
        ga.graph = t
        res = ga.run(ga.generate_population(), id=f"{len(t):03d}:{j:03d}")
        print(res["max_fitness"])
logger.save("./output/test_new.csv")


Test Case 0
44.0
44.0
38.0
43.0
44.0
43.0
44.0
38.0
40.0
44.0
Test Case 1
29.0
33.0
40.0
29.0
32.0
36.0
33.0
31.0
33.0
31.0
Test Case 2
16.0
20.0
18.0
19.0
15.0
19.0
22.0
18.0
19.0
21.0
Test Case 0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
Test Case 1
47.0
46.0
44.0
48.0
50.0
48.0
48.0
48.0
48.0
46.0
Test Case 2
29.0
30.0
30.0
28.0
28.0
29.0
28.0
29.0
28.0
29.0
